# ASMSA: Run molecular dynamics with CV derived AAE model

**Previous steps**
- [prepare.ipynb](prepare.ipynb): Download and sanity check input files
- [train.ipynb](train.ipynb): Use results of previous tuning in more thorough training|
- [tune.ipynb](tune.ipynb): Perform initial hyperparameter tuning for this molecule



In [ ]:
# %cd villin

In [ ]:
!cp ../*.mdp .

In [1]:
#avoid TF to consume GPU memory
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
tf.config.list_logical_devices()

2025-04-14 21:08:31.486011: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-14 21:08:31.499929: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-14 21:08:31.504053: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 21:08:31.514785: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 21:08:32.271044: W tensorflow/compiler/tf2

[LogicalDevice(name='/device:CPU:0', device_type='CPU')]

In [2]:
import asmsa
import os
import re
import numpy as np
import matplotlib.pyplot as plt
import gromacs as gmx

NOTE: Some configuration directories are not set up yet: 
	/home/jovyan/.gromacswrapper
	/home/jovyan/.gromacswrapper/qscripts
	/home/jovyan/.gromacswrapper/templates
NOTE: You can create the configuration file and directories with:
	>>> import gromacs
	>>> gromacs.config.setup()


pod/asmsa-gmx-34032-5d22x condition met
pod/asmsa-gmx-34032-fbzt8 condition met


In [3]:
ompthreads=4
mpiranks=1
gpus=1

In [4]:
# Kubernetes deployment
mdrunner=gmx.MDrunnerK8s()

def mdrun(**kwargs):
    mdrunner.run(pre={'omp':ompthreads,'mpi':mpiranks,'gpus':gpus}, mdrunargs={**kwargs,'ntomp':ompthreads,'pin':'on'},ncores=mpiranks)


In [5]:
# alternative local deployment
'''
mdrunner=gmx.run.MDrunner()

# XXX: no MPI support so far
def mdrun(**kwargs):
    mdrunner.run(mdrunargs={**kwargs,'ntomp':ompthreads,'pin':'on'})
'''

"\nmdrunner=gmx.run.MDrunner()\n\n# XXX: no MPI support so far\ndef mdrun(**kwargs):\n    mdrunner.run(mdrunargs={**kwargs,'ntomp':ompthreads,'pin':'on'})\n"

In [6]:
exec(open('inputs.py').read())

fulltop = base + '-full.top'

In [7]:
# on fresh run only!
!cp {topol} {fulltop}

In [8]:
# Dodecahedron box of suitable size (adjust eventually)

mdbox=2.0
gmx.editconf(f=gro,o=f"{base}-box.gro",c=True,d=str(mdbox),bt="dodecahedron")

pod/asmsa-gmx-34032-5d22x condition met
pod/asmsa-gmx-34032-fbzt8 condition met


Note that major changes are planned in future for editconf, to improve usability and utility.
Read 272 atoms
Volume: 79.9182 nm^3, corresponds to roughly 35900 electrons
No velocities found
    system size :  1.786  2.082  1.574 (nm)
    diameter    :  2.245               (nm)
    center      :  2.159  2.162  2.161 (nm)
    box vectors :  4.307  4.307  4.307 (nm)
    box angles  :  90.00  90.00  90.00 (degrees)
    box volume  :  79.92               (nm^3)
    shift       :  2.525  2.522  0.047 (nm)
new center      :  4.684  4.684  2.208 (nm)
new box vectors :  6.245  6.245  6.245 (nm)
new box angles  :  60.00  60.00  90.00 (degrees)
new box volume  : 172.25               (nm^3)


            :-) GROMACS - gmx editconf, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx editconf -f ../ASMSA.final/trpcage_npt400_nH.gro -o ../ASMSA.final/trpcage_npt400_nH-box.gro -c -d 2.0 -bt dodecahedron


GROMACS reminds you: "Take away paradox from the thinker and you have a professor." (Soren Kirkegaard)



(0, None, None)

In [9]:
# Solvate
gmx.solvate(cp=f"{base}-box.gro",cs="spc216.gro",o=f"{base}-solv.gro",p=fulltop)

pod/asmsa-gmx-34032-5d22x condition met
pod/asmsa-gmx-34032-fbzt8 condition met
            :-) GROMACS - gmx solvate, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx solvate -cp ../ASMSA.final/trpcage_npt400_nH-box.gro -cs spc216.gro -o ../ASMSA.final/trpcage_npt400_nH-solv.gro -p ../ASMSA.final/trpcage_npt400_nH-full.top

Reading solute configuration
Reading solvent configuration

Initialising inter-atomic distances...
Generating solvent configuration
Will generate new solvent configuration of 4x4x3 boxes
Solvent box contains 20307 atoms in 6769 residues
Removed 3384 solvent atoms due to solvent-solvent overlap
Removed 258 solvent atoms due to solute-solvent overlap
Sorting configuration
Found 1 molecule type:
    SOL (   3 atoms):  5555 residues
Generated solvent containing 16665 atoms in 5555 residues
Writing generated configuration to ../ASMSA.final/trpcage_npt400_nH-solv.gro




         based on residue and atom names, since they could not be
         definitively assigned from the information in your input
         files. These guessed numbers might deviate from the mass
         and radius of the atom type. Please check the output
         files if necessary. Note, that this functionality may
         be removed in a future GROMACS version. Please, consider
         using another file format for your input.

NOTE: From version 5.0 gmx solvate uses the Van der Waals radii
from the source below. This means the results may be different
compared to previous GROMACS versions.

++++ PLEASE READ AND CITE THE FOLLOWING REFERENCE ++++
A. Bondi
van der Waals Volumes and Radii
J. Phys. Chem. 68 (1964) pp. 441-451
-------- -------- --- Thank You --- -------- --------

Adding line for 5555 solvent molecules with resname (SOL) to topology file (../ASMSA.final/trpcage_npt400_nH-full.top)


(0, None, None)

In [10]:
# Add ions
gmx.grompp(f='trpcage_md_data/ions.mdp',c=f"{base}-solv.gro",p=fulltop,o="ions.tpr")

pod/asmsa-gmx-34032-5d22x condition met
pod/asmsa-gmx-34032-fbzt8 condition met


Setting the LD random seed to -70274

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein'

Excluding 2 bonded neighbours molecule type 'SOL'
Analysing residue names:
There are:    20    Protein residues
There are:  5555      Water residues
Analysing Protein...

This run will generate roughly 1 Mb of data


             :-) GROMACS - gmx grompp, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx grompp -f trpcage_md_data/ions.mdp -c ../ASMSA.final/trpcage_npt400_nH-solv.gro -p ../ASMSA.final/trpcage_npt400_nH-full.top -o ions.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file trpcage_md_data/ions.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5

NOTE 2 [file trpcage_npt400_nH-full.top, line 2701]:
  System has non-zero total charge: -1.000000
  Total charge should normally be an integer. See
  http://www.gromacs.org/Documentation/Floating_Point_Arithmetic
  for discussion on how close it should be to an integer.



Number of degrees of freedom in T-Coupling group rest is 34143.00
The integrator does not provide a ensemble t

(0, None, None)

In [11]:
gmx.select(s="ions.tpr",on='sol.ndx',select='SOL')
gmx.genion(s="ions.tpr",n="sol.ndx",o=f"{base}-ions.gro",p=fulltop,pname="NA",nname="CL",neutral=True)

pod/asmsa-gmx-34032-5d22x condition met
pod/asmsa-gmx-34032-fbzt8 condition met
             :-) GROMACS - gmx select, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx select -s ions.tpr -on sol.ndx -select SOL

Reading file ions.tpr, VERSION 2023.2-plumed_2.10.0_dev (single precision)
Reading file ions.tpr, VERSION 2023.2-plumed_2.10.0_dev (single precision)
Analyzed topology coordinates

GROMACS reminds you: "There is just one thing I can promise you about the outer-space program: your tax dollar will go farther." (Wernher von Braun)

pod/asmsa-gmx-34032-5d22x condition met
pod/asmsa-gmx-34032-fbzt8 condition met


Will try to add 1 NA ions and 0 CL ions.
Select a continuous group of solvent molecules

Processing topology
Replacing 1 solute molecules in topology file (../ASMSA.final/trpcage_npt400_nH-full.top)  by 1 NA and 0 CL ions.


             :-) GROMACS - gmx genion, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx genion -s ions.tpr -n sol.ndx -o ../ASMSA.final/trpcage_npt400_nH-ions.gro -p ../ASMSA.final/trpcage_npt400_nH-full.top -pname NA -nname CL -neutral

Reading file ions.tpr, VERSION 2023.2-plumed_2.10.0_dev (single precision)
Reading file ions.tpr, VERSION 2023.2-plumed_2.10.0_dev (single precision)
Group     0 (            SOL) has 16665 elements
There is one group in the index
Number of (3-atomic) solvent molecules: 5555

Back Off! I just backed up ../ASMSA.final/trpcage_npt400_nH-full.top to ../ASMSA.final/#trpcage_npt400_nH-full.top.2#
Using random seed -8389410.
Replacing solvent molecule 5498 (atom 16766) with NA


GROMACS reminds you: "You Could Make More Money As a Butcher" (F. Zappa)



(0, None, None)

In [12]:
# Energy minimization

#gmx.start(f"grompp -f minim-sol.mdp -c {base}-ions.gro -p {base}.top -o em.tpr",wait=True,delete=True)
gmx.grompp(f="trpcage_md_data/minim-sol.mdp",c=f"{base}-ions.gro",p=fulltop,o="em.tpr")

pod/asmsa-gmx-34032-5d22x condition met
pod/asmsa-gmx-34032-fbzt8 condition met


Setting the LD random seed to -193609810

Generated 2145 of the 2145 non-bonded parameter combinations

Generated 2145 of the 2145 1-4 parameter combinations

Excluding 3 bonded neighbours molecule type 'Protein'

Excluding 2 bonded neighbours molecule type 'SOL'

Excluding 1 bonded neighbours molecule type 'NA'
Analysing residue names:
There are:    20    Protein residues
There are:  5554      Water residues
There are:     1        Ion residues
Analysing Protein...

The largest distance between excluded atoms is 0.398 nm between atom 58 and 75
Calculating fourier grid dimensions for X Y Z
Using a fourier grid of 56x56x56, spacing 0.112 0.112 0.112

Estimate for the relative computational load of the PME mesh part: 0.30

This run will generate roughly 1 Mb of data


             :-) GROMACS - gmx grompp, 2023.2-plumed_2.10.0_dev (-:

Executable:   /gromacs/AVX2_256_ts/bin/gmx
Data prefix:  /gromacs/AVX2_256_ts
Working dir:  /mnt/ASMSA
Command line:
  gmx grompp -f trpcage_md_data/minim-sol.mdp -c ../ASMSA.final/trpcage_npt400_nH-ions.gro -p ../ASMSA.final/trpcage_npt400_nH-full.top -o em.tpr

Ignoring obsolete mdp entry 'ns_type'

NOTE 1 [file trpcage_md_data/minim-sol.mdp]:
  With Verlet lists the optimal nstlist is >= 10, with GPUs >= 20. Note
  that with the Verlet scheme, nstlist has no effect on the accuracy of
  your simulation.

Generating 1-4 interactions: fudge = 0.5
Number of degrees of freedom in T-Coupling group rest is 34140.00
The integrator does not provide a ensemble temperature, there is no system ensemble temperature

There was 1 NOTE

GROMACS reminds you: "We all understand the twinge of discomfort at the thought that we share a common ancestor with the apes. No one can embarrass you like a relative." (Neal DeGrasse Tyson)



(0, None, None)

In [15]:
# This job may run a bit longer, start it in background, with more cores and GPU

#gmx.start(f"mdrun -v -deffnm em -pin on",cores=4,gpus=1)
mdrun(deffnm="em")

job.batch/gmx-e25bda9f-016b-4cb9-9a86-bcb1cdc1ba64 created


error: no matching resources found


job.batch "gmx-e25bda9f-016b-4cb9-9a86-bcb1cdc1ba64" deleted


Unable to use a TTY - input is not a terminal or the right kind of file
Error from server (BadRequest): pod gmx-e25bda9f-016b-4cb9-9a86-bcb1cdc1ba64-8crs8 does not have a host assigned


In [ ]:
# Isothermal-isochoric equilibration, following the same job pattern
gmx.grompp(f="trpcage_md_data/nvt.mdp",c="em.gro",r="em.gro",p=fulltop,o="nvt.tpr")

In [ ]:
mdrun(deffnm='nvt')

In [ ]:
# And isothermal-isobaric equilibration
gmx.grompp(f='npt.mdp',c='nvt.gro',r='nvt.gro',t='nvt.cpt',p=fulltop,o='npt.tpr')

In [ ]:
mdrun(deffnm='npt')

In [ ]:
# Extract temperature, pressure and density from the trajectories and plot them
gmx.energy(f='npt.edr',o='press.xvg',input='Pressure')
gmx.energy(f='npt.edr',o='dens.xvg',input='Density')
gmx.energy(f='npt.edr',o='temp.xvg',input='Temperature')

In [ ]:
temp = np.loadtxt('temp.xvg',comments=['#','@'])
press = np.loadtxt('press.xvg',comments=['#','@'])
dens = np.loadtxt('dens.xvg',comments=['#','@'])

plt.figure(figsize=(15,9))
plt.subplot(311)
plt.plot(press[:,0],press[:,1])
plt.title('isothermal-isobaric equilibration')
plt.grid()
#plt.xlabel('time (ps)')
plt.ylabel("pressure (bar)")

plt.subplot(312)
plt.ylabel('density (kg/m3)')
plt.grid()
plt.plot(dens[:,0],dens[:,1])

plt.subplot(313)
plt.xlabel('time (ps)')
plt.ylabel('temperature (K)')
plt.grid()
plt.plot(temp[:,0],temp[:,1])

plt.show()

In [ ]:
# Now we are ready to run metadynamics with plumed.dat generated in train.ipynb
# Checkpoints are stored, hence the simulation can be restarted if it gets killed
#1 m

#mdsteps=500000000

# 500 ns
mdsteps=250000000

# 10 ns
#mdsteps=5000000

# 100 ps
#mdsteps=50000

In [ ]:
with open('md.mdp','w') as mdp:
    mdp.write(f'''
integrator              = md        ; leap-frog integrator
dt                      = 0.002     ; 2 fs
; Output control
nstxout                 = 0         ; suppress bulky .trr file by specifying 
nstvout                 = 0         ; 0 for output frequency of nstxout,
nstfout                 = 0         ; nstvout, and nstfout
nstenergy               = 5000      ; save energies every 10.0 ps
nstlog                  = 5000      ; update log file every 10.0 ps
nstxout-compressed      = 5000      ; save compressed coordinates every 10.0 ps
; compressed-x-grps       = System    ; save the whole system
compressed-x-grps = Protein
; Bond parameters
continuation            = yes       ; Restarting after NPT 
constraint_algorithm    = lincs     ; holonomic constraints 
constraints             = h-bonds   ; bonds involving H are constrained
lincs_iter              = 1         ; accuracy of LINCS
lincs_order             = 4         ; also related to accuracy
; Neighborsearching
cutoff-scheme           = Verlet    ; Buffered neighbor searching
ns_type                 = grid      ; search neighboring grid cells
nstlist                 = 10        ; 20 fs, largely irrelevant with Verlet scheme
rcoulomb                = 1.0       ; short-range electrostatic cutoff (in nm)
rvdw                    = 1.0       ; short-range van der Waals cutoff (in nm)
; Electrostatics
coulombtype             = PME       ; Particle Mesh Ewald for long-range electrostatics
pme_order               = 4         ; cubic interpolation
fourierspacing          = 0.16      ; grid spacing for FFT
; Temperature coupling is on
tcoupl                  = V-rescale             ; modified Berendsen thermostat
tc-grps                 = Protein Non-Protein   ; two coupling groups - more accurate
tau_t                   = 0.1     0.1           ; time constant, in ps
ref_t                   = 300     300           ; reference temperature, one for each group, in K
; Pressure coupling is on
pcoupl                  = Parrinello-Rahman     ; Pressure coupling on in NPT
pcoupltype              = isotropic             ; uniform scaling of box vectors
tau_p                   = 2.0                   ; time constant, in ps
ref_p                   = 1.0                   ; reference pressure, in bar
compressibility         = 4.5e-5                ; isothermal compressibility of water, bar^-1
; Periodic boundary conditions
pbc                     = xyz       ; 3-D PBC
; Dispersion correction
DispCorr                = EnerPres  ; account for cut-off vdW scheme
; Velocity generation
gen_vel                 = no        ; Velocity generation is off 

nsteps = {mdsteps}
''')

In [ ]:
gmx.grompp(f='md.mdp',c='trpcage_md_data/npt.gro',t='trpcage_md_data/npt.cpt',p=fulltop,o='md.tpr')

In [ ]:
!touch HILLS
#mdrun(deffnm='md',plumed='plumed.dat',cpi='md.cpt')
#mdrun(deffnm='md')

In [ ]:
mdrunner.prehook(mpi=4,omp=4,gpus=1)

In [ ]:
gmx.trjconv(f='md.xtc',s=base+'-box.gro',pbc='nojump',input='Protein Protein'.split(),o='pbc.xtc')

In [ ]:
gmx.trjconv(f='pbc.xtc',s=base+'-box.gro',fit='rot+trans',input='Protein Protein'.split(),o='fit.xtc')

In [ ]:
import mdtraj as md
tr = md.load('fit.xtc',top=base+'-box.gro')

In [ ]:
import nglview as nv
nv.show_mdtraj(tr)

In [ ]:
mdrunner.prehook(mpi=2,omp=4,gpus=1)

In [ ]:
gmx.convert_tpr(s='md.tpr',o='md-new.tpr',extend=100000)
!mv md-new.tpr md.tpr

In [ ]:
mdrun(deffnm='md',plumed='plumed.dat',cpi='md.cpt')